# Interacting with LLMs from various providers using Haystack 

## Connecting to OpenAI's GPT models

In [1]:
from haystack.preview import Pipeline
from haystack.preview.components.builders.prompt_builder import PromptBuilder
from haystack.preview.components.generators.openai.gpt import GPTGenerator
from haystack.preview.components.generators.hugging_face.hugging_face_local import HuggingFaceLocalGenerator
import os
from dotenv import load_dotenv
load_dotenv("./../../.env")

# Open AI API Key
openai_key = os.getenv("OPENAI_API_KEY")


In [2]:
def initialize_simple_pipeline(llm_generator, llm_generator_name, prompt_template):
    # Creating a pipeline
    pipeline = Pipeline()

    # Adding a PromptBuilder
    prompt_builder = PromptBuilder(template=prompt_template)
    pipeline.add_component(instance=prompt_builder, name="prompt_builder")

    # Adding a GPT-based Generator
    # Ensure that you have the OPENAI_API_KEY environment variable set
    gpt_generator = llm_generator # GPTGenerator(api_key=os.environ.get("OPENAI_API_KEY"))
    pipeline.add_component(instance=gpt_generator, name=llm_generator_name) #"gpt_generator")

    # Connecting the components
    pipeline.connect("prompt_builder",llm_generator_name)

    return pipeline

In [ ]:
prompt_template = """
    Given a question, you provide with an accurate answer using Shakespeare's style.

    \nQuestion: {{question}}
    \nAnswer:
    """


In [ ]:
gpt = GPTGenerator(api_key=os.getenv("OPENAI_API_KEY"),  
                                                  model_name='gpt-4', 
                                                  api_base_url = 'https://api.openai.com/v1',
                                                  )

In [ ]:
gpt.run("Who lives in Paris?")

In [ ]:

# Initializing the pipeline
simple_pipeline = initialize_simple_pipeline(llm_generator=gpt, llm_generator_name="gpt_generator", prompt_template=prompt_template)

# Running a question through the pipeline
question = "What is the capital of France?"
result = simple_pipeline.run(
    {
        "prompt_builder": {"question": question},
    }
)

# Printing the result
print(result)


In [ ]:
simple_pipeline.draw( "gpt-pipeline.png")


## Connecting to Local HuggingFace LLM

In [3]:
local_generator = HuggingFaceLocalGenerator(model_name_or_path="bigscience/bloom-3b",
                                          task="text-generation",
                                          generation_kwargs={
                                            "max_new_tokens": 1000,
                                            "temperature": 0.1,
                                            "do_sample":  True
                                            })

In [5]:
local_generator.warm_up()

In [6]:
answer = local_generator.run("Who lives in Paris?")

## Custom component: connecting to Mistral using a custom component

In [ ]:
from scripts.huggingfaceendpoints import HuggingFaceModelQuery
import os

In [ ]:
hugging_face_token_endpoint = os.getenv("mistral_hf_token")
hf_url = "https://g683oq4smsqs4s4b.us-east-1.aws.endpoints.huggingface.cloud"

mistral = HuggingFaceModelQuery(api_url=hf_url, api_key=hugging_face_token_endpoint, parameters={
    "temperature": 0.01,
    "max_length": 100
})

mistral.run("Answer using one word: What is the Capital of France")

In [ ]:
prompt_template = """
    Given a question, answer using one word.

    \nQuestion: {{question}}
    \nAnswer:
    """

In [ ]:
prompt_template = """
    Given a question, you provide with a helpful answer using Shakespeare's language.

    \nQuestion: {{question}}
    \nAnswer:
    """
def initialize_simple_pipeline(hf_url, hugging_face_token_endpoint):
    # Creating a pipeline
    pipeline = Pipeline()

    # Adding a PromptBuilder
    prompt_builder = PromptBuilder(template=prompt_template)
    pipeline.add_component(instance=prompt_builder, name="prompt_builder")

    # Adding a GPT-based Generator
    # Ensure that you have the OPENAI_API_KEY environment variable set
    mistral = HuggingFaceModelQuery(api_url=hf_url, api_key=hugging_face_token_endpoint, parameters={
            "temperature": 0.1,
            "max_length": 500
        })
    pipeline.add_component(instance=mistral, name="mistral")

    # Connecting the components
    pipeline.connect("prompt_builder", "mistral")

    return pipeline

# Initializing the pipeline
simple_pipeline = initialize_simple_pipeline(hf_url=hf_url, hugging_face_token_endpoint=hugging_face_token_endpoint)



In [ ]:
simple_pipeline.draw( "mistral-pipeline.png")

In [ ]:
question = "What is the capital of France?"
result = simple_pipeline.run(
    {
        "prompt_builder": {"question": question},
    }
)

# Printing the result
print(result)
